In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [2]:
excel_path = "/Users/jakobbengtsson/Desktop/testdata_exjobb.xlsx"
indata_sheet = "indata"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.1,1.21
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.2,1.44
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.3,1.69
4,5,Dealer,Deal4,4,10,1,0.95,NBD,1.0,1.4,1.96
5,6,Dealer,Deal5,5,10,1,0.95,NBD,1.0,1.5,2.25


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [8]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance,Lead time demand mean,Lead time demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,Gamma,5.0,1.302872,NaN,50.0,13.028722
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,1.21,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,1.44,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,1.69,NaN,NaN
4,5,Dealer,Deal4,4,10,1,0.95,NBD,1.0,1.400000,1.96,NaN,NaN
5,6,Dealer,Deal5,5,10,1,0.95,NBD,1.0,1.500000,2.25,NaN,NaN


RDC reorder-point optimization

In [5]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
outdataDF.loc[outdataDF["Type"] == "RDC", "Beta"] = beta_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Beta"] = beta_arr
print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



Optimal weighted induced backorder cost at the warehouse is: 0.7093481387222141, betas are: [0.36044571 0.73766048 0.98969763 0.74560762 0.71332925]


In [6]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)
outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0
outdataDF


Starting optimizing, R = 0, c = nan, c+1 = nan


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance,Lead time demand mean,Lead time demand stdev,Beta,R optimal
0,1,RDC,Johannesburg,10,40,1,NaN,Gamma,5.0,1.302872,NaN,50.0,13.028722,0.709348,0.0
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,1.21,NaN,NaN,0.360446,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,1.44,NaN,NaN,0.737660,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,1.69,NaN,NaN,0.989698,NaN
4,5,Dealer,Deal4,4,10,1,0.95,NBD,1.0,1.400000,1.96,NaN,NaN,0.745608,NaN
5,6,Dealer,Deal5,5,10,1,0.95,NBD,1.0,1.500000,2.25,NaN,NaN,0.713329,NaN


In [7]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance,Lead time demand mean,Lead time demand stdev,Beta,R optimal,Waiting time,Lead time
0,1,RDC,Johannesburg,10,40,1,NaN,Gamma,5.0,1.302872,NaN,50.0,13.028722,0.709348,0.0,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,1.21,10.0,11.000000,0.360446,NaN,0.0,10.0
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,1.44,3.0,3.600000,0.737660,NaN,0.0,3.0
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,1.69,2.0,2.600000,0.989698,NaN,0.0,2.0
4,5,Dealer,Deal4,4,10,1,0.95,NBD,1.0,1.400000,1.96,4.0,5.600000,0.745608,NaN,0.0,4.0
5,6,Dealer,Deal5,5,10,1,0.95,NBD,1.0,1.500000,2.25,5.0,7.500000,0.713329,NaN,0.0,5.0


In [8]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

NBD 1.0 1.1
NBD 1.0 1.2
NBD 1.0 1.3
NBD 1.0 1.4
NBD 1.0 1.5


In [9]:
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance,Lead time demand mean,Lead time demand stdev,Beta,R optimal,Waiting time,Lead time,Realized item fill rate,Expected stock on hand
0,1,RDC,Johannesburg,10,40,1,NaN,Gamma,5.0,1.302872,NaN,50.0,13.028722,0.709348,0.0,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,1.21,10.0,11.000000,0.360446,13.0,0.0,10.0,0.955594,8.563821
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,1.44,3.0,3.600000,0.737660,5.0,0.0,3.0,0.970201,7.523758
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,1.69,2.0,2.600000,0.989698,4.0,0.0,2.0,0.971491,7.519877
4,5,Dealer,Deal4,4,10,1,0.95,NBD,1.0,1.400000,1.96,4.0,5.600000,0.745608,7.0,0.0,4.0,0.959639,8.546174
5,6,Dealer,Deal5,5,10,1,0.95,NBD,1.0,1.500000,2.25,5.0,7.500000,0.713329,9.0,0.0,5.0,0.957720,9.558558


In [11]:
W

0.0

In [12]:
rdc_f_u_probability_array

array([nan])